<a href="https://colab.research.google.com/github/Jasper-Hewitt/Wind_project/blob/main/1_by_1_mom_327.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# init

## get and split data

In [1]:
#todo little update. how to push
#fix the max number of tokens, it's actually very little
#補充 answer with the rest of the rows
#make a better while loop that creates different dataframes based on word count, code now a bit ugly
!pip install fitz
!pip install static
!pip install frontend
!pip install pymupdf --upgrade
!pip install openai

  Using cached fitz-0.0.1.dev2-py2.py3-none-any.whl (20 kB)
  Using cached configobj-5.0.8-py2.py3-none-any.whl (36 kB)
  Using cached configparser-5.3.0-py3-none-any.whl (19 kB)
  Using cached nibabel-5.1.0-py3-none-any.whl (3.3 MB)
  Using cached nipype-1.8.6-py3-none-any.whl (3.2 MB)
  Using cached pyxnat-1.5-py3-none-any.whl (95 kB)
  Using cached scipy-1.10.1-cp311-cp311-macosx_12_0_arm64.whl (28.7 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached prov-2.0.0-py3-none-any.whl (421 kB)
  Using cached pydot-1.4.2-py2.py3-none-any.whl (21 kB)
  Using cached rdflib-6.3.2-py3-none-any.whl (528 kB)
  Using cached simplejson-3.19.1-cp311-cp311-macosx_11_0_arm64.whl (74 kB)
  Using cached traits-6.3.2-cp311-cp311-macosx_10_9_universal2.whl
  Using cached etelemetry-0.3.0-py3-none-any.whl (6.3 kB)
  Using cached looseversion-1.1.2-py3-none-any.whl (8.1 kB)
  Using cached pathlib-1.0.1-py3-none-any.whl (14 kB)
  Using cached future-0.18.3-py3-none-any.whl
  Using cache

In [ ]:
import os
import openai
import json

openai.api_key = '' #delete when uploading 

In [ ]:
#manually download the data from github and upload it in this notebook
import fitz

doc = fitz.open("mom_327.pdf")

for page in doc:
    text = page.get_text("text")
    with open("output3.txt", "a") as f:
        f.write(text)

In [ ]:
with open('output3.txt', 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

In [ ]:
import re
pattern = "。" #。|:
split_contents = re.split(pattern, contents)
# print(sentences)

In [ ]:
split_contents

In [ ]:
len(split_contents)

241

# question 1: 第327次會議是什麼時候？


## sentiment search 1： 第327次會議是什麼時候？

In [ ]:
#sentiment search
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
corpus = split_contents
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# add queries, between 3 and 5
queries = ['第327次會議是什麼時候？'] #'第327次會議中，對於鯨豚保育的重點是什麼？', '第327次會議中，有哪些專家為鯨豚保育發言？']

# Find the closest X sentences of the corpus for each query sentence based on cosine similarity
top_k = min(68, len(corpus))

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop X most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("Sentence: ", corpus[idx], " Original Sequence: ", idx+1, " (Score: {:.4f})".format(score))

In [ ]:
len(queries)

1

In [ ]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [ ]:
len(sentence_scores)

240

In [ ]:
#adjust to also allow for queries outside the range of 3-5
import pandas as pd

if len(queries)==1:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]]))

elif len(queries)==2:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[0]]+scores[queries[1]]))

elif len(queries)==3:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]))
elif len(queries)==4:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]))
elif len(queries)==5:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[4]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]+scores[queries[4]])) 

#________________________________________________________________________________________________________________________________________________________________________________________________________

if len(queries) == 1:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 2:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 3:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 4:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 5:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'similarity_score5', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

In [ ]:
df

,text,original_sequence,accumulated_score
0,1 \n行政院環境保護署環境影響評估審查委員會 \n第 327 次會議紀錄 \n壹、 時間：...,1,tensor(0.5738)
1,\n伍、 確認出席委員已達法定人數後，主席致詞：略,2,tensor(0.5060)
144,\n八、 彰化縣線西鄉公所 \n（一） 為節能減碳及提升效率，本所書面意見即視同開會出席\...,145,tensor(0.4335)
21,\n18 \n4. 建議目的事業主管機關經濟部能源局辦理以下事項： \n（1） 依 106...,22,tensor(0.4082)
26,\n5. 本環境影響說明書定稿經本署備查後始得動工，並應於\n開發行為施工前 30 日內，...,27,tensor(0.3854)
...,...,...,...
235,\n八、 空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高...,236,tensor(-0.0720)
80,調查結\n果如下，經評估開發行為對稀有植物及保育類動物\n無顯著不利影響： \n○1 陸域植...,81,tensor(-0.0759)
35,調查結果如下，經評估開發行\n為對稀有植物及保育類動物無顯著不利影響： \n（1） 陸域植物...,36,tensor(-0.0855)
36,\n（2） 陸域動物：陸域哺乳類、兩棲類、爬蟲類、蝴蝶與\n蜻蜓類均無保育類物種,37,tensor(-0.0967)


In [ ]:
noise_pattern= '遵照辦理'
noise_pattern1='章節'
noise_pattern2='��'
noise_pattern3='�'
noise_pattern4='修訂處'
noise_pattern5='頁次'

df['text'] = df['text'].replace(noise_pattern,'',regex=True)
df['text'] = df['text'].replace(noise_pattern1,'',regex=True)
df['text'] = df['text'].replace(noise_pattern2,'',regex=True)
df['text'] = df['text'].replace(noise_pattern3,'',regex=True)
df['text'] = df['text'].replace(noise_pattern4,'',regex=True)
df['text'] = df['text'].replace(noise_pattern5,'',regex=True)

In [ ]:
#delete empty rows
df = df[df['text'].str.len()>6]   
#the line below no longer makes sense lol. 
# df = df[['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score']]
df = df[['text', 'original_sequence', 'accumulated_score']]
df = df.reset_index(drop=True)
df

,text,original_sequence,accumulated_score
0,1 \n行政院環境保護署環境影響評估審查委員會 \n第 327 次會議紀錄 \n壹、 時間：...,1,tensor(0.5738)
1,\n伍、 確認出席委員已達法定人數後，主席致詞：略,2,tensor(0.5060)
2,\n八、 彰化縣線西鄉公所 \n（一） 為節能減碳及提升效率，本所書面意見即視同開會出席\...,145,tensor(0.4335)
3,\n18 \n4. 建議目的事業主管機關經濟部能源局辦理以下事項： \n（1） 依 106...,22,tensor(0.4082)
4,\n5. 本環境影響說明書定稿經本署備查後始得動工，並應於\n開發行為施工前 30 日內，...,27,tensor(0.3854)
...,...,...,...
234,\n八、 空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高...,236,tensor(-0.0720)
235,調查結\n果如下，經評估開發行為對稀有植物及保育類動物\n無顯著不利影響： \n○1 陸域植...,81,tensor(-0.0759)
236,調查結果如下，經評估開發行\n為對稀有植物及保育類動物無顯著不利影響： \n（1） 陸域植物...,36,tensor(-0.0855)
237,\n（2） 陸域動物：陸域哺乳類、兩棲類、爬蟲類、蝴蝶與\n蜻蜓類均無保育類物種,37,tensor(-0.0967)


In [ ]:
#I used sentiment search to judge whether GPTs output is related to our keywords but I ran into a problem: there is no way to check whether the information is complete. For example, if we set the default threshold
#to 0.7000, and then ask chatGPT to make a summary about certain topics within that text, it will only make a summary about those topics and leave the rest#####
#also, if start with a really high threshold (which may only lead to 2 or 3 sentences) chat gpt will write a perfect summary that SBERT will judge with a high tensor score. This means that the threshold will not go down
#and much information that may also be related to the subject is being left out.

#I therefore thought of an alternative way to create a dynamic threshold. In the code below we find the last row, so the lowest possible tensor score, that contains our 關鍵詞 and delete all the rows after that.
#this way we can be sure that all the important sentences related to our topic will be taken into consideration by GPT for writing its summary or answering our questions. 

# Find the index of the last row containing the word "鯨豚"
index = df[df['text'].str.contains('會議')][::-1].index[0]

# Drop all of the rows after the last row containing the word "鯨豚"
new_df = df.loc[:index]

In [ ]:
new_df

,text,original_sequence,accumulated_score
0,1 \n行政院環境保護署環境影響評估審查委員會 \n第 327 次會議紀錄 \n壹、 時間：...,1,tensor(0.5738)
1,\n伍、 確認出席委員已達法定人數後，主席致詞：略,2,tensor(0.5060)
2,\n八、 彰化縣線西鄉公所 \n（一） 為節能減碳及提升效率，本所書面意見即視同開會出席\...,145,tensor(0.4335)
3,\n18 \n4. 建議目的事業主管機關經濟部能源局辦理以下事項： \n（1） 依 106...,22,tensor(0.4082)
4,\n5. 本環境影響說明書定稿經本署備查後始得動工，並應於\n開發行為施工前 30 日內，...,27,tensor(0.3854)
...,...,...,...
68,\n六、 電纜路線規劃 \n \n72 \n（一） 彰化地區海纜上岸路線優先規劃於台灣電力...,233,tensor(0.1968)
69,\n7. 內政部依海岸管理法訂定或公布之「一、二級海岸保護\n區」,193,tensor(0.1965)
70,\n3. 簡報 P.10 打樁期間承諾「不使用聲學裝置(ADD)」，請\n納入報告書第 8...,156,tensor(0.1949)
71,\n6. 統籌彰化縣外海通過環境影響評估審查之各離岸風力\n發電案件，於 106 年秋季至...,113,tensor(0.1934)


### determine threshold

In [ ]:
#we will have to consider how we want to determine the threshold, the simplest way would be to look at the dataframe and check from where it doesn't have anything to do 
#with 噪音 anymore. if there is a more mathematical way to solve this problem I am also open to that. 

# threshold = 0.7710
# mask = df['accumulated_score'].apply(lambda x: x.item() > threshold)

# sbert_out = df[mask]
# sbert_out

### sort by original sequence (optional)

In [ ]:
# sbert_out = sbert_out.sort_values(by='original_sequence').reset_index(drop=True)
# sbert_out=sbert_out['text']

In [ ]:
# sbert_out=sbert_out.to_frame()
# sbert_out

## Summary with GPT 

### prepare data

In [ ]:
#disable this cell for sort by orginal sequence 
#fix this this is super confusing lol!!!!
sbert_out=new_df
sbert_out=sbert_out['text']  
sbert_out=sbert_out.to_frame() 
df=sbert_out

In [ ]:
df      

,text
0,1 \n行政院環境保護署環境影響評估審查委員會 \n第 327 次會議紀錄 \n壹、 時間：...
1,\n伍、 確認出席委員已達法定人數後，主席致詞：略
2,\n八、 彰化縣線西鄉公所 \n（一） 為節能減碳及提升效率，本所書面意見即視同開會出席\...
3,\n18 \n4. 建議目的事業主管機關經濟部能源局辦理以下事項： \n（1） 依 106...
4,\n5. 本環境影響說明書定稿經本署備查後始得動工，並應於\n開發行為施工前 30 日內，...
...,...
68,\n六、 電纜路線規劃 \n \n72 \n（一） 彰化地區海纜上岸路線優先規劃於台灣電力...
69,\n7. 內政部依海岸管理法訂定或公布之「一、二級海岸保護\n區」
70,\n3. 簡報 P.10 打樁期間承諾「不使用聲學裝置(ADD)」，請\n納入報告書第 8...
71,\n6. 統籌彰化縣外海通過環境影響評估審查之各離岸風力\n發電案件，於 106 年秋季至...


In [ ]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 1500
index = df.index[df['cumulative_length'] > 1500]

if index.size > 0:
    index = index[0] - 1
else:
    index = df.shape[0] - 1

# Create a new dataframe with only the last few rows
new_df = df.iloc[index+1:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index+1]
df

In [ ]:
df

,text,text_length,cumulative_length
0,1 \n行政院環境保護署環境影響評估審查委員會 \n第 327 次會議紀錄 \n壹、 時間：...,176,176
1,\n伍、 確認出席委員已達法定人數後，主席致詞：略,25,201
2,\n八、 彰化縣線西鄉公所 \n（一） 為節能減碳及提升效率，本所書面意見即視同開會出席\...,192,393
3,\n18 \n4. 建議目的事業主管機關經濟部能源局辦理以下事項： \n（1） 依 106...,107,500
4,\n5. 本環境影響說明書定稿經本署備查後始得動工，並應於\n開發行為施工前 30 日內，...,106,606
5,\n（五） 4 案開發行為依環境影響評估法施行細則第 19 條第 1 項\n第 1 款及第...,428,1034


In [ ]:
sbert_out=df['text']
# sbert_out_1=new_df['text']
# sbert_out_1 = sbert_out_1.reset_index(drop=True)
# sbert_out.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
# sbert_out_1.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
sbert_out=sbert_out.to_frame()


In [ ]:
sbert_out

,text
0,1 \n行政院環境保護署環境影響評估審查委員會 \n第 327 次會議紀錄 \n壹、 時間：...
1,\n伍、 確認出席委員已達法定人數後，主席致詞：略
2,\n八、 彰化縣線西鄉公所 \n（一） 為節能減碳及提升效率，本所書面意見即視同開會出席\...
3,\n18 \n4. 建議目的事業主管機關經濟部能源局辦理以下事項： \n（1） 依 106...
4,\n5. 本環境影響說明書定稿經本署備查後始得動工，並應於\n開發行為施工前 30 日內，...
5,\n（五） 4 案開發行為依環境影響評估法施行細則第 19 條第 1 項\n第 1 款及第...


In [ ]:
# sbert_text = sbert_out['text'].to_string(index=False) #this leads to incomplete text. use join
sbert_text = " ".join(sbert_out['text'].tolist())
len(sbert_text)

1039

In [ ]:
sbert_text = re.sub(r'[\n\\]+', '', sbert_text)
sbert_text= re.sub(r'n|n－', '', sbert_text)
sbert_text

'1 行政院環境保護署環境影響評估審查委員會 第 327 次會議紀錄 壹、 時間：107 年 2 月 9 日（星期五）下午 4 時 30 分 貳、 地點：本署 4 樓第 5 會議室 參、 主席：李主任委員應元（詹副主任委員順貴代）                          記錄：劉彥均 肆、 出（列）席單位及人員：如後附會議簽名單  伍、 確認出席委員已達法定人數後，主席致詞：略  八、 彰化縣線西鄉公所 （一） 為節能減碳及提升效率，本所書面意見即視同開會出席意見，卻屢未見目的事業主管機關經濟部能源局書面回應或來電與本所討論未同意確認之事由，本案後續如比照辦理，請確認是否依 106 年 11 月 27 日 4 案專案小組第 3 次聯席初審會議結論（三）略以：「經有關委員及相關機關確認後，提行政院環境保護署環境影響評估審查委員會討論」結論辦理  18 4. 建議目的事業主管機關經濟部能源局辦理以下事項： （1） 依 106 年 7 月 19 日本署環境影響評估審查委員會第 316 次會議決議，協助於本案施工前建立後續開發行為第三方監測及觀測機制  5. 本環境影響說明書定稿經本署備查後始得動工，並應於開發行為施工前 30 日內，以書面告知目的事業主管機關及本署預定施工日期；採分段（分期）開發者，則提報各段（期）開發之第 1 次施工行為預定施工日期  （五） 4 案開發行為依環境影響評估法施行細則第 19 條第 1 項第 1 款及第 2 款各目所列情形逐項檢討如下，併 106 年11 月 27 日 4 案專案小組第 3 次聯席初審會議結論及前述修正意見提委員會討論： 1. 就 4 案開發行為包括環境影響評估法施行細則第 19 條第 1 項第 1 款附表二之「345 千伏或 161 千伏輸電線路架空或地下化線路鋪設長度 50 公里以上者」，考量開發單位採行高電壓輸出海纜，減少海纜鋪設數量或範 圍，施工方式除潮間帶採水平導向式鑽掘(HDD)，其餘海纜範圍採犁埋機或噴埋機，配合海纜鋪設完成後海床沉積物隨即自然覆蓋，開發單位承諾依「離岸風電區塊開發政策評估說明書」本署徵詢意見採行因應對策，海纜上岸路線規劃於台灣電力股份有限公司依經濟部 106年 8 月 2 日經能字第 10602611030 號函公告「彰化離岸風電海纜上岸共同廊道範圍」之北側廊道，以減輕整體環境影響'

In [ ]:
len(sbert_text)

998

### prompt GPT

In [ ]:
summary = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '第327次會議是什麼時候？'+sbert_text, #add: 如果找不到答案，請跟我說
  temperature=0.9,
  max_tokens=300,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary = summary + text

summary

'，並應於建置完成後每年進行第三方監測觀測及相關報告，故此案可於本署環境影響評估審查委員會討論得以通過 \n\n第327次會議的時間是107年2月9日（星期五）下午4點30分。'

In [ ]:
print('第327次會議是什麼時候？'+ summary)
# print('第327次會議中，對於鯨豚保育的重點是什麼？'+ summary1)
# print('第327次會議中，有哪些專家為鯨豚保育發言？'+ summary2)

第327次會議是什麼時候？，並應於建置完成後每年進行第三方監測觀測及相關報告，故此案可於本署環境影響評估審查委員會討論得以通過 

第327次會議的時間是107年2月9日（星期五）下午4點30分。


# question 2: 第327次會議中，對於鯨豚保育的重點是什麼？

## sentiment search 2

In [ ]:
#sentiment search
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
corpus = split_contents
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# add queries, between 1 and 5
queries = ['第327次會議中，對於鯨豚保育的重點是什麼？'] #, '第327次會議中，有哪些專家為鯨豚保育發言？']

# Find the closest X sentences of the corpus for each query sentence based on cosine similarity
top_k = min(68, len(corpus))

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop X most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("Sentence: ", corpus[idx], " Original Sequence: ", idx+1, " (Score: {:.4f})".format(score))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/




Query: 第327次會議中，對於鯨豚保育的重點是什麼？

Top X most similar sentences in corpus:
Sentence:  」劉委員希平意見略以：「2 點意
見提供經濟部能源局參考，第 1，鑑於航道外側風場較
遠平時不容易觀察，故施工期間應加強監督開發單位對
鯨豚之觀測是否確實，實際上水中生物除鯨豚外，尚有
許多應保護之其他生物  Original Sequence:  tensor(67)  (Score: 0.6966)
Sentence:   
（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅
限於 4～9 月執行，調整前應依法申請變更），建議強化
鯨豚觀測員訓練作業，並考量邀民間團體具鯨豚觀測能
力人員共同參與  Original Sequence:  tensor(217)  (Score: 0.6756)
Sentence:   
（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺
發現有鯨豚母子對或瀕臨絕種類保育類鯨豚之案
件，應繼續辦理 4 季合計至少 30 趟次之鯨豚調查
作業，並提出環境影響調查報告送審  Original Sequence:  tensor(210)  (Score: 0.6756)
Sentence:   
3. 行政院農業委員會依野生動物保育法訂定之「野生動物
重要棲息環境」「野生動物保護區」「中華白海豚野生
動物重要棲息環境（含預告）」  Original Sequence:  tensor(190)  (Score: 0.6609)
Sentence:   
○4 鯨豚：4 案風場均非位於中華白海豚野生動物重
要棲息環境預告範圍，並依水下噪音模擬評估結
果，已擬定海豚保護措施  Original Sequence:  tensor(86)  (Score: 0.6158)
Sentence:   
（4） 鯨豚：4 案風場均非位於中華白海豚野生動物重要
棲息環境預告範圍，並依水下噪音模擬評估結果，
已擬定海豚保護措施  Original Sequence:

In [ ]:
type(queries)

list

In [ ]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [ ]:
len(sentence_scores)

240

In [ ]:
#adjust to also allow for queries outside the range of 3-5
import pandas as pd

if len(queries)==1:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]]))

elif len(queries)==2:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[0]]+scores[queries[1]]))

elif len(queries)==3:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]))
elif len(queries)==4:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]))
elif len(queries)==5:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[4]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]+scores[queries[4]])) 

#________________________________________________________________________________________________________________________________________________________________________________________________________

if len(queries) == 1:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 2:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 3:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 4:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 5:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'similarity_score5', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

In [ ]:
df

,text,original_sequence,accumulated_score
65,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...,66,tensor(0.6966)
215,\n（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅\n限於 4～9 月執行...,216,tensor(0.6756)
208,"\n（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨...",209,tensor(0.6756)
188,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...,189,tensor(0.6609)
84,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,85,tensor(0.6158)
...,...,...,...
45,\n7. 開發計畫屬潔淨再生能源風力發電，營運階段於機組\n運轉期間僅以天然風力提供機組運...,46,tensor(0.0789)
90,\n（7） 開發計畫屬潔淨再生能源風力發電，營運階段於機\n組運轉期間僅以天然風力提供機組...,91,tensor(0.0737)
47,\n9. 開發計畫屬潔淨再生能源風力發電，營運階段於機組\n運轉期間僅以天然風力提供機組運...,48,tensor(0.0573)
235,\n八、 空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高...,236,tensor(0.0348)


In [ ]:
noise_pattern= '遵照辦理'
noise_pattern1='章節'
noise_pattern2='��'
noise_pattern3='�'
noise_pattern4='修訂處'
noise_pattern5='頁次'

df['text'] = df['text'].replace(noise_pattern,'',regex=True)
df['text'] = df['text'].replace(noise_pattern1,'',regex=True)
df['text'] = df['text'].replace(noise_pattern2,'',regex=True)
df['text'] = df['text'].replace(noise_pattern3,'',regex=True)
df['text'] = df['text'].replace(noise_pattern4,'',regex=True)
df['text'] = df['text'].replace(noise_pattern5,'',regex=True)

In [ ]:
#delete empty rows
df = df[df['text'].str.len()>6]   
#the line below no longer makes sense lol. 
# df = df[['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score']]
df = df[['text', 'original_sequence', 'accumulated_score']]
df = df.reset_index(drop=True)
df

,text,original_sequence,accumulated_score
0,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...,66,tensor(0.6966)
1,\n（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅\n限於 4～9 月執行...,216,tensor(0.6756)
2,"\n（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨...",209,tensor(0.6756)
3,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...,189,tensor(0.6609)
4,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,85,tensor(0.6158)
...,...,...,...
234,\n7. 開發計畫屬潔淨再生能源風力發電，營運階段於機組\n運轉期間僅以天然風力提供機組運...,46,tensor(0.0789)
235,\n（7） 開發計畫屬潔淨再生能源風力發電，營運階段於機\n組運轉期間僅以天然風力提供機組...,91,tensor(0.0737)
236,\n9. 開發計畫屬潔淨再生能源風力發電，營運階段於機組\n運轉期間僅以天然風力提供機組運...,48,tensor(0.0573)
237,\n八、 空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高...,236,tensor(0.0348)


In [ ]:
#I used sentiment search to judge whether GPTs output is related to our keywords but I ran into a problem: there is no way to check whether the information is complete. For example, if we set the default threshold
#to 0.7000, and then ask chatGPT to make a summary about certain topics within that text, it will only make a summary about those topics and leave the rest#####
#also, if start with a really high threshold (which may only lead to 2 or 3 sentences) chat gpt will write a perfect summary that SBERT will judge with a high tensor score. This means that the threshold will not go down
#and much information that may also be related to the subject is being left out.

#I therefore thought of an alternative way to create a dynamic threshold. In the code below we find the last row, so the lowest possible tensor score, that contains our 關鍵詞 and delete all the rows after that.
#this way we can be sure that all the important sentences related to our topic will be taken into consideration by GPT for writing its summary or answering our questions. 

# Find the index of the last row containing the word "鯨豚"
index = df[df['text'].str.contains('鯨豚')][::-1].index[0]

# Drop all of the rows after the last row containing the word "鯨豚"
new_df = df.loc[:index]

In [ ]:
new_df

,text,original_sequence,accumulated_score
0,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...,66,tensor(0.6966)
1,\n（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅\n限於 4～9 月執行...,216,tensor(0.6756)
2,"\n（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨...",209,tensor(0.6756)
3,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...,189,tensor(0.6609)
4,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,85,tensor(0.6158)
...,...,...,...
207,\n（四） 依據環保署公告「營建工程噪音評估模式技術規範」之\nSoundPlan 模式模...,178,tensor(0.1822)
208,於風機退役之前，將與政府單\n位做密切溝通，同時參考即將在未來幾年要執行之歐洲\n大型風場除役經驗,184,tensor(0.1789)
209,\n（八） 風場除役計畫納入循環經濟理念，除役時所有取出的零\n組件及物品將運送至選定的港...,183,tensor(0.1767)
210,"\n（七） P.5-7 說明南北向風機間距 1,950 公尺至 2,400 公尺，經\n檢...",169,tensor(0.1734)


## Summary with GPT 2

### prepare data

In [ ]:
#disable this cell for sort by orginal sequence 
#fix this this is super confusing lol!!!!
sbert_out=new_df
sbert_out=sbert_out['text']  
sbert_out=sbert_out.to_frame() 
df=sbert_out

In [ ]:
df      

,text
0,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...
1,\n（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅\n限於 4～9 月執行...
2,"\n（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨..."
3,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...
4,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...
...,...
207,\n（四） 依據環保署公告「營建工程噪音評估模式技術規範」之\nSoundPlan 模式模...
208,於風機退役之前，將與政府單\n位做密切溝通，同時參考即將在未來幾年要執行之歐洲\n大型風場除役經驗
209,\n（八） 風場除役計畫納入循環經濟理念，除役時所有取出的零\n組件及物品將運送至選定的港...
210,"\n（七） P.5-7 說明南北向風機間距 1,950 公尺至 2,400 公尺，經\n檢..."


In [ ]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 1500
index = df.index[df['cumulative_length'] > 1500]

if index.size > 0:
    index = index[0] - 1
else:
    index = df.shape[0] - 1

# Create a new dataframe with only the last few rows
new_df = df.iloc[index+1:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index+1]
df

In [ ]:
df

,text,text_length,cumulative_length
0,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...,103,103
1,\n（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅\n限於 4～9 月執行...,94,197
2,"\n（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨...",98,295
3,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...,69,364
4,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,61,425
5,\n（4） 鯨豚：4 案風場均非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...,62,487
6,\n3. 施工船上配置至少 3 位以上之鯨豚觀測員（至少 1 位為\n民間生態團體成員）於...,101,588
7,\n（二） 針對前項特定保護對象需納入緩衝帶規劃，建議基座位\n址需距離中華白海豚野生動物...,66,654
8,\n（五） 就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等\n保育類野生動物之規劃，恐衍...,70,724
9,\n（八） 第 5 章已承諾使用管架式(Jacket)基礎，建議納入 8.1.1.1\n節...,55,779


In [ ]:
sbert_out=df['text']
# sbert_out_1=new_df['text']
# sbert_out_1 = sbert_out_1.reset_index(drop=True)
# sbert_out.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
# sbert_out_1.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
sbert_out=sbert_out.to_frame()


In [ ]:
sbert_out

,text
0,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...
1,\n（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅\n限於 4～9 月執行...
2,"\n（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨..."
3,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...
4,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...
5,\n（4） 鯨豚：4 案風場均非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...
6,\n3. 施工船上配置至少 3 位以上之鯨豚觀測員（至少 1 位為\n民間生態團體成員）於...
7,\n（二） 針對前項特定保護對象需納入緩衝帶規劃，建議基座位\n址需距離中華白海豚野生動物...
8,\n（五） 就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等\n保育類野生動物之規劃，恐衍...
9,\n（八） 第 5 章已承諾使用管架式(Jacket)基礎，建議納入 8.1.1.1\n節...


In [ ]:
# sbert_text = sbert_out['text'].to_string(index=False) #this leads to incomplete text. use join
sbert_text = " ".join(sbert_out['text'].tolist())
len(sbert_text)

1502

In [ ]:
sbert_text = re.sub(r'[\n\\]+', '', sbert_text)
sbert_text= re.sub(r'n|n－', '', sbert_text)
sbert_text

'」劉委員希平意見略以：「2 點意見提供經濟部能源局參考，第 1，鑑於航道外側風場較遠平時不容易觀察，故施工期間應加強監督開發單位對鯨豚之觀測是否確實，實際上水中生物除鯨豚外，尚有許多應保護之其他生物  （七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅限於 4～9 月執行，調整前應依法申請變更），建議強化鯨豚觀測員訓練作業，並考量邀民間團體具鯨豚觀測能力人員共同參與  （2） 環境影響評估書件記載風場區域及外圍 1,500 公尺發現有鯨豚母子對或瀕臨絕種類保育類鯨豚之案件，應繼續辦理 4 季合計至少 30 趟次之鯨豚調查作業，並提出環境影響調查報告送審  3. 行政院農業委員會依野生動物保育法訂定之「野生動物重要棲息環境」「野生動物保護區」「中華白海豚野生動物重要棲息環境（含預告）」  ○4 鯨豚：4 案風場均非位於中華白海豚野生動物重要棲息環境預告範圍，並依水下噪音模擬評估結果，已擬定海豚保護措施  （4） 鯨豚：4 案風場均非位於中華白海豚野生動物重要棲息環境預告範圍，並依水下噪音模擬評估結果，已擬定海豚保護措施  3. 施工船上配置至少 3 位以上之鯨豚觀測員（至少 1 位為民間生態團體成員）於基礎打樁過程同時目視觀察，觀察範圍必須涵蓋 4 個方位之警戒區（750 公尺內）和預警區（1,500 公尺內）  （二） 針對前項特定保護對象需納入緩衝帶規劃，建議基座位址需距離中華白海豚野生動物重要棲息環境（含預告）1,000 公尺以上  （五） 就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等保育類野生動物之規劃，恐衍生疑慮，建議暫緩採用，宜審慎蒐集案例研析後再行考量  （八） 第 5 章已承諾使用管架式(Jacket)基礎，建議納入 8.1.1.1節二、鯨豚減輕對策（一）  2. 於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察船及配置鯨豚生態觀察員，於基礎打樁過程持續監測 」劉委員小如意見略以：「第 1，簡報 P.8鯨豚影響減輕對策中，請確認不得啟動新設風機打樁作業時間為『日落前 1 小時後至日出前』還是『日落前 2小時至日出前』？第 2，簡報 P.13 針對開發單位所提『撞擊影響』『屏障效應』及『覓食地喪失』等 3 項鳥類生態監測指標之對策，表列內容均屬於欲執行之工作項目，並非對策，另所述『未來如調查及分析結果有環境傷害而無適合之減輕對策情形

In [ ]:
len(sbert_text)

1438

### prompt GPT

In [ ]:
summary1 = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= ' 第327次會議中，對於鯨豚保育的重點是什麼？'+sbert_text, #add: 如果找不到答案，請跟我說
  temperature=0.9,
  max_tokens=900,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary1 = summary1 + text

summary1

'」 \n\n第327次會議中，對於鯨豚保育的重點是加強監督開發單位對鯨豚之觀測；施工期間需每年至少30趟次之鯨豚生態調查；施工船上應配置至少3位以上之鯨豚觀測員，並於基礎打樁過程同時目視觀察；施工前應先確認至少30分鐘無鯨豚活動；工作範圍必須涵蓋4個方位之警戒區（750公尺）和預警區（1,500公尺），船速應管制在6節以下，並儘可能避免在中華白海豚活動高峰時間進入已知之中華白海豚活動密集位置。'

In [ ]:
#print('第327次會議是什麼時候？'+ summary)
print('第327次會議中，對於鯨豚保育的重點是什麼？'+ summary1)
# print('第327次會議中，有哪些專家為鯨豚保育發言？'+ summary2)

第327次會議中，對於鯨豚保育的重點是什麼？」 

第327次會議中，對於鯨豚保育的重點是加強監督開發單位對鯨豚之觀測；施工期間需每年至少30趟次之鯨豚生態調查；施工船上應配置至少3位以上之鯨豚觀測員，並於基礎打樁過程同時目視觀察；施工前應先確認至少30分鐘無鯨豚活動；工作範圍必須涵蓋4個方位之警戒區（750公尺）和預警區（1,500公尺），船速應管制在6節以下，並儘可能避免在中華白海豚活動高峰時間進入已知之中華白海豚活動密集位置。


#question 3:  第327次會議中，有哪些專家為鯨豚保育發言？

## sentiment search 3

In [ ]:
#sentiment search
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
corpus = split_contents
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# add queries, between 3 and 5
queries = ['第327次會議中，有哪些專家為鯨豚保育發言？']

# Find the closest X sentences of the corpus for each query sentence based on cosine similarity
top_k = min(68, len(corpus))

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop X most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("Sentence: ", corpus[idx], " Original Sequence: ", idx+1, " (Score: {:.4f})".format(score))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/




Query: 第327次會議中，有哪些專家為鯨豚保育發言？

Top X most similar sentences in corpus:
Sentence:  」劉委員希平意見略以：「2 點意
見提供經濟部能源局參考，第 1，鑑於航道外側風場較
遠平時不容易觀察，故施工期間應加強監督開發單位對
鯨豚之觀測是否確實，實際上水中生物除鯨豚外，尚有
許多應保護之其他生物  Original Sequence:  tensor(67)  (Score: 0.6858)
Sentence:   
（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺
發現有鯨豚母子對或瀕臨絕種類保育類鯨豚之案
件，應繼續辦理 4 季合計至少 30 趟次之鯨豚調查
作業，並提出環境影響調查報告送審  Original Sequence:  tensor(210)  (Score: 0.6650)
Sentence:   
（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅
限於 4～9 月執行，調整前應依法申請變更），建議強化
鯨豚觀測員訓練作業，並考量邀民間團體具鯨豚觀測能
力人員共同參與  Original Sequence:  tensor(217)  (Score: 0.6479)
Sentence:   
3. 行政院農業委員會依野生動物保育法訂定之「野生動物
重要棲息環境」「野生動物保護區」「中華白海豚野生
動物重要棲息環境（含預告）」  Original Sequence:  tensor(190)  (Score: 0.6264)
Sentence:   
○4 鯨豚：4 案風場均非位於中華白海豚野生動物重
要棲息環境預告範圍，並依水下噪音模擬評估結
果，已擬定海豚保護措施  Original Sequence:  tensor(86)  (Score: 0.6001)
Sentence:   
（4） 鯨豚：4 案風場均非位於中華白海豚野生動物重要
棲息環境預告範圍，並依水下噪音模擬評估結果，
已擬定海豚保護措施  Original Sequence:

In [ ]:
len(queries)

1

In [ ]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [ ]:
len(sentence_scores)

240

In [ ]:
#adjust to also allow for queries outside the range of 3-5
import pandas as pd

if len(queries)==1:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]]))

elif len(queries)==2:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[0]]+scores[queries[1]]))

elif len(queries)==3:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]))
elif len(queries)==4:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]))
elif len(queries)==5:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[4]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]+scores[queries[4]])) 

#________________________________________________________________________________________________________________________________________________________________________________________________________

if len(queries) == 1:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 2:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 3:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 4:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 5:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'similarity_score5', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

In [ ]:
df

,text,original_sequence,accumulated_score
65,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...,66,tensor(0.6858)
208,"\n（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨...",209,tensor(0.6650)
215,\n（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅\n限於 4～9 月執行...,216,tensor(0.6479)
188,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...,189,tensor(0.6264)
84,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,85,tensor(0.6001)
...,...,...,...
10,\n（2） 日落前1小時後至日出前不得啟動新設風機打樁作\n業,11,tensor(0.0657)
90,\n（7） 開發計畫屬潔淨再生能源風力發電，營運階段於機\n組運轉期間僅以天然風力提供機組...,91,tensor(0.0596)
235,\n八、 空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高...,236,tensor(0.0457)
47,\n9. 開發計畫屬潔淨再生能源風力發電，營運階段於機組\n運轉期間僅以天然風力提供機組運...,48,tensor(0.0076)


In [ ]:
noise_pattern= '遵照辦理'
noise_pattern1='章節'
noise_pattern2='��'
noise_pattern3='�'
noise_pattern4='修訂處'
noise_pattern5='頁次'

df['text'] = df['text'].replace(noise_pattern,'',regex=True)
df['text'] = df['text'].replace(noise_pattern1,'',regex=True)
df['text'] = df['text'].replace(noise_pattern2,'',regex=True)
df['text'] = df['text'].replace(noise_pattern3,'',regex=True)
df['text'] = df['text'].replace(noise_pattern4,'',regex=True)
df['text'] = df['text'].replace(noise_pattern5,'',regex=True)

In [ ]:
#delete empty rows
df = df[df['text'].str.len()>6]   
#the line below no longer makes sense lol. 
# df = df[['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score']]
df = df[['text', 'original_sequence', 'accumulated_score']]
df = df.reset_index(drop=True)
df

,text,original_sequence,accumulated_score
0,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...,66,tensor(0.6858)
1,"\n（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨...",209,tensor(0.6650)
2,\n（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅\n限於 4～9 月執行...,216,tensor(0.6479)
3,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...,189,tensor(0.6264)
4,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,85,tensor(0.6001)
...,...,...,...
234,\n（2） 日落前1小時後至日出前不得啟動新設風機打樁作\n業,11,tensor(0.0657)
235,\n（7） 開發計畫屬潔淨再生能源風力發電，營運階段於機\n組運轉期間僅以天然風力提供機組...,91,tensor(0.0596)
236,\n八、 空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高...,236,tensor(0.0457)
237,\n9. 開發計畫屬潔淨再生能源風力發電，營運階段於機組\n運轉期間僅以天然風力提供機組運...,48,tensor(0.0076)


In [ ]:
#I used sentiment search to judge whether GPTs output is related to our keywords but I ran into a problem: there is no way to check whether the information is complete. For example, if we set the default threshold
#to 0.7000, and then ask chatGPT to make a summary about certain topics within that text, it will only make a summary about those topics and leave the rest#####
#also, if start with a really high threshold (which may only lead to 2 or 3 sentences) chat gpt will write a perfect summary that SBERT will judge with a high tensor score. This means that the threshold will not go down
#and much information that may also be related to the subject is being left out.

#I therefore thought of an alternative way to create a dynamic threshold. In the code below we find the last row, so the lowest possible tensor score, that contains our 關鍵詞 and delete all the rows after that.
#this way we can be sure that all the important sentences related to our topic will be taken into consideration by GPT for writing its summary or answering our questions. 

# Find the index of the last row containing the word "鯨豚"
index = df[df['text'].str.contains('會議')][::-1].index[0]

# Drop all of the rows after the last row containing the word "鯨豚"
new_df = df.loc[:index]

In [ ]:
new_df

,text,original_sequence,accumulated_score
0,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...,66,tensor(0.6858)
1,"\n（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨...",209,tensor(0.6650)
2,\n（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅\n限於 4～9 月執行...,216,tensor(0.6479)
3,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...,189,tensor(0.6264)
4,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,85,tensor(0.6001)
...,...,...,...
133,至於本署 106 年受理審查經濟部（能源\n局）轉送環評案件，為達降低風場離岸過近衍生環境疑...,199,tensor(0.3150)
134,\n（2） 陸域動物：陸域哺乳類、兩棲類、爬蟲類、蝴蝶與\n蜻蜓類均無保育類物種,37,tensor(0.3139)
135,\n9. 於陸域自設升（降）壓站及纜線施工開挖期間，委請文\n化資產考古人員進行跟隨監看,105,tensor(0.3137)
136,\n（4） 洽科技部、文化部及教育部等有關部會，藉由我國\n離岸風機發展之契機，建構水下文...,25,tensor(0.3130)


## Summary with GPT 3

### prepare data

In [ ]:
#disable this cell for sort by orginal sequence 
#fix this this is super confusing lol!!!!
sbert_out=new_df
sbert_out=sbert_out['text']  
sbert_out=sbert_out.to_frame() 
df=sbert_out

In [ ]:
df      

,text
0,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...
1,"\n（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨..."
2,\n（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅\n限於 4～9 月執行...
3,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...
4,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...
...,...
133,至於本署 106 年受理審查經濟部（能源\n局）轉送環評案件，為達降低風場離岸過近衍生環境疑...
134,\n（2） 陸域動物：陸域哺乳類、兩棲類、爬蟲類、蝴蝶與\n蜻蜓類均無保育類物種
135,\n9. 於陸域自設升（降）壓站及纜線施工開挖期間，委請文\n化資產考古人員進行跟隨監看
136,\n（4） 洽科技部、文化部及教育部等有關部會，藉由我國\n離岸風機發展之契機，建構水下文...


In [ ]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 1500
index = df.index[df['cumulative_length'] > 1500]

if index.size > 0:
    index = index[0] - 1
else:
    index = df.shape[0] - 1

# Create a new dataframe with only the last few rows
new_df = df.iloc[index+1:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index+1]
df

In [ ]:
df

,text,text_length,cumulative_length
0,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...,103,103
1,"\n（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨...",98,201
2,\n（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅\n限於 4～9 月執行...,94,295
3,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...,69,364
4,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,61,425
5,\n（4） 鯨豚：4 案風場均非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...,62,487
6,\n（八） 第 5 章已承諾使用管架式(Jacket)基礎，建議納入 8.1.1.1\n節...,55,542
7,\n3. 施工船上配置至少 3 位以上之鯨豚觀測員（至少 1 位為\n民間生態團體成員）於...,101,643
8,\n（五） 就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等\n保育類野生動物之規劃，恐衍...,70,713
9,」劉委員小如意見略以：「第 1，簡報 P.8\n鯨豚影響減輕對策中，請確認不得啟動新設風機打...,333,1046


In [ ]:
sbert_out=df['text']
# sbert_out_1=new_df['text']
# sbert_out_1 = sbert_out_1.reset_index(drop=True)
# sbert_out.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
# sbert_out_1.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
sbert_out=sbert_out.to_frame()


In [ ]:
sbert_out

,text
0,」劉委員希平意見略以：「2 點意\n見提供經濟部能源局參考，第 1，鑑於航道外側風場較\n遠...
1,"\n（2） 環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨..."
2,\n（七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅\n限於 4～9 月執行...
3,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...
4,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...
5,\n（4） 鯨豚：4 案風場均非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...
6,\n（八） 第 5 章已承諾使用管架式(Jacket)基礎，建議納入 8.1.1.1\n節...
7,\n3. 施工船上配置至少 3 位以上之鯨豚觀測員（至少 1 位為\n民間生態團體成員）於...
8,\n（五） 就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等\n保育類野生動物之規劃，恐衍...
9,」劉委員小如意見略以：「第 1，簡報 P.8\n鯨豚影響減輕對策中，請確認不得啟動新設風機打...


In [ ]:
# sbert_text = sbert_out['text'].to_string(index=False) #this leads to incomplete text. use join
sbert_text = " ".join(sbert_out['text'].tolist())
len(sbert_text)

1457

In [ ]:
sbert_text = re.sub(r'[\n\\]+', '', sbert_text)
sbert_text= re.sub(r'n|n－', '', sbert_text)
sbert_text

'」劉委員希平意見略以：「2 點意見提供經濟部能源局參考，第 1，鑑於航道外側風場較遠平時不容易觀察，故施工期間應加強監督開發單位對鯨豚之觀測是否確實，實際上水中生物除鯨豚外，尚有許多應保護之其他生物  （2） 環境影響評估書件記載風場區域及外圍 1,500 公尺發現有鯨豚母子對或瀕臨絕種類保育類鯨豚之案件，應繼續辦理 4 季合計至少 30 趟次之鯨豚調查作業，並提出環境影響調查報告送審  （七） 施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅限於 4～9 月執行，調整前應依法申請變更），建議強化鯨豚觀測員訓練作業，並考量邀民間團體具鯨豚觀測能力人員共同參與  3. 行政院農業委員會依野生動物保育法訂定之「野生動物重要棲息環境」「野生動物保護區」「中華白海豚野生動物重要棲息環境（含預告）」  ○4 鯨豚：4 案風場均非位於中華白海豚野生動物重要棲息環境預告範圍，並依水下噪音模擬評估結果，已擬定海豚保護措施  （4） 鯨豚：4 案風場均非位於中華白海豚野生動物重要棲息環境預告範圍，並依水下噪音模擬評估結果，已擬定海豚保護措施  （八） 第 5 章已承諾使用管架式(Jacket)基礎，建議納入 8.1.1.1節二、鯨豚減輕對策（一）  3. 施工船上配置至少 3 位以上之鯨豚觀測員（至少 1 位為民間生態團體成員）於基礎打樁過程同時目視觀察，觀察範圍必須涵蓋 4 個方位之警戒區（750 公尺內）和預警區（1,500 公尺內）  （五） 就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等保育類野生動物之規劃，恐衍生疑慮，建議暫緩採用，宜審慎蒐集案例研析後再行考量 」劉委員小如意見略以：「第 1，簡報 P.8鯨豚影響減輕對策中，請確認不得啟動新設風機打樁作業時間為『日落前 1 小時後至日出前』還是『日落前 2小時至日出前』？第 2，簡報 P.13 針對開發單位所提『撞擊影響』『屏障效應』及『覓食地喪失』等 3 項鳥類生態監測指標之對策，表列內容均屬於欲執行之工作項目，並非對策，另所述『未來如調查及分析結果有環境傷害而無適合之減輕對策情形，將與監督委員會研商可能之對策及復育補償』並不適當，如未來對環境有傷害而無減輕對策，應立即評估環境是否能承受該傷害，如可接受，再與監督委員會研商可行之復育補償對策，反之，若無法承受，則在既無適合之減輕對策亦無法復育補償下，環境必須無條件

In [ ]:
len(sbert_text)

1396

### prompt GPT

In [ ]:
 #remove key when uploading to github
summary2 = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '第327次會議中，有哪些專家為鯨豚保育發言？'+sbert_text, #add: 如果找不到答案，請跟我說
  temperature=0.9,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary2 = summary2 + text

summary2

'，請增加具體說明」。\n\n專家為鯨豚保育發言的有：\n\n- 劉委員希平\n- 劉委員小如意\n- 鄭委員明修'

In [ ]:
#print('第327次會議是什麼時候？'+ summary)
# print('第327次會議中，對於鯨豚保育的重點是什麼？'+ summary1)
print('第327次會議中，有哪些專家為鯨豚保育發言？'+ summary2)

第327次會議中，有哪些專家為鯨豚保育發言？，請增加具體說明」。

專家為鯨豚保育發言的有：

- 劉委員希平
- 劉委員小如意
- 鄭委員明修


# final output (answers to all three questions)

In [ ]:
print('第327次會議是什麼時候？'+ summary)
print('第327次會議中，對於鯨豚保育的重點是什麼？'+ summary1)
print('第327次會議中，有哪些專家為鯨豚保育發言？'+ summary2)

第327次會議是什麼時候？，並應於建置完成後每年進行第三方監測觀測及相關報告，故此案可於本署環境影響評估審查委員會討論得以通過 

第327次會議的時間是107年2月9日（星期五）下午4點30分。
第327次會議中，對於鯨豚保育的重點是什麼？」 

第327次會議中，對於鯨豚保育的重點是加強監督開發單位對鯨豚之觀測；施工期間需每年至少30趟次之鯨豚生態調查；施工船上應配置至少3位以上之鯨豚觀測員，並於基礎打樁過程同時目視觀察；施工前應先確認至少30分鐘無鯨豚活動；工作範圍必須涵蓋4個方位之警戒區（750公尺）和預警區（1,500公尺），船速應管制在6節以下，並儘可能避免在中華白海豚活動高峰時間進入已知之中華白海豚活動密集位置。
第327次會議中，有哪些專家為鯨豚保育發言？，請增加具體說明」。

專家為鯨豚保育發言的有：

- 劉委員希平
- 劉委員小如意
- 鄭委員明修


In [ ]:
allQandA='第327次會議是什麼時候？'+summary +'第327次會議中，對於鯨豚保育的重點是什麼？'+ summary1+'第327次會議中，有哪些專家為鯨豚保育發言？'+ summary2

In [ ]:
allQandA

'第327次會議是什麼時候？，並應於建置完成後每年進行第三方監測觀測及相關報告，故此案可於本署環境影響評估審查委員會討論得以通過 \n\n第327次會議的時間是107年2月9日（星期五）下午4點30分。第327次會議中，對於鯨豚保育的重點是什麼？」 \n\n第327次會議中，對於鯨豚保育的重點是加強監督開發單位對鯨豚之觀測；施工期間需每年至少30趟次之鯨豚生態調查；施工船上應配置至少3位以上之鯨豚觀測員，並於基礎打樁過程同時目視觀察；施工前應先確認至少30分鐘無鯨豚活動；工作範圍必須涵蓋4個方位之警戒區（750公尺）和預警區（1,500公尺），船速應管制在6節以下，並儘可能避免在中華白海豚活動高峰時間進入已知之中華白海豚活動密集位置。第327次會議中，有哪些專家為鯨豚保育發言？，請增加具體說明」。\n\n專家為鯨豚保育發言的有：\n\n- 劉委員希平\n- 劉委員小如意\n- 鄭委員明修'

In [ ]:
final = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '請給我總結一下之下的內容，總結應該包括所有問題的答案，包括日期，還有專家是誰'+allQandA, #add: 如果找不到答案，請跟我說
  temperature=0.9,
  max_tokens=1300,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
final = final + text

final

'\n- 鄭委員明治\n- 林委員權明\n\n總結：第327次會議的時間是107年2月9日（星期五）下午4點30分。會議中針對鯨豚保育的重點為：加強開發單位對鯨豚之觀測，施工期間每年至少30趟次之鯨豚生態調查，施工船上配置至少3位以上之鯨豚觀測員，施工前確認30分鐘無鯨豚活動，工作範圍必須涵蓋4個方位之警戒區（750公尺）及預警區（1,500公尺），船速應管制在6節以下，並儘可能避免在中華白海豚活動高峰時間進入已知之中華白海豚活動密集位置，並應於建置完成後每年進行第三方監測觀測及相關報告，故此案可於本署環境影響評估審查委員會討論得以通過，會議中專家為鯨豚保育發言的有劉委員希平、劉委員小如意、鄭委員明修、鄭委員明治及林委員權明。'

## (optional) adjust answers based on more information (not finished yet)

### prepare additional data

In [ ]:
new_df

In [ ]:
# 補充answer with other rows?
# reset index
new_df=new_df['text']
new_df = new_df.reset_index(drop=True)


In [ ]:
new_df=new_df.to_frame()

In [ ]:
df=new_df

In [ ]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 3600
index = df.index[df['cumulative_length'] > 3600][0]

# Create a new dataframe with only the last few rows
new_df = df.iloc[index:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index]

In [ ]:
df

In [ ]:
sbert_out_1=df['text']
sbert_out_1=sbert_out_1.to_frame()
sbert_out_1

In [ ]:
sbert_text_1 = sbert_out_1['text'].to_string(index=False)

In [ ]:
sbert_text_1 = re.sub(r'[\n\\]+', '', sbert_text_1)
sbert_text_1 = re.sub(r'n|n－', '', sbert_text_1)
sbert_text_1

### prompt GPT (part 2)